In [ ]:
# %pip install scikit-learn
# %pip install tensorflow
# %pip install seaborn matplotlib pandas

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import os
from tensorflow.keras.backend import clear_session
from sklearn.model_selection import train_test_split
import shutil
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Dense, Flatten
from tensorflow.keras.models import load_model
from tensorflow.keras import mixed_precision
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, average_precision_score
# Enable mixed precision training
mixed_precision.set_global_policy('mixed_float16')

import warnings
warnings.filterwarnings('ignore')

clear_session()  # This frees the GPU memory, peventing training deadlock

# Function to create directories
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)
        
setz = ['metal','glass', 'trash', 'cardboard', 'paper', 'plastic']

for setClass in [setz[2]]: #setz:
    clear_session()
    global classez
    classez = [setClass, 'other']
    # data path
    global dataset_path
    dataset_path = f'/app/data/Datasets/augmented_{classez[0]}_and_{classez[1]}'  # <- docker file path      # "G:\My Drive\Datasets\ "
    

    tar = (224, 224)
    batch = 16 # 16
    #Split the dataset
    # Create an ImageDataGenerator for each subset
    datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # 20% for validation

    # Training data generator
    train_generator = datagen.flow_from_directory(
        dataset_path,
        target_size=tar,
        batch_size=batch,
        class_mode='binary',
        seed=42,
        subset='training')  # Set as training data

    # Validation data generator
    validation_generator = datagen.flow_from_directory(
        dataset_path,
        target_size=tar,
        batch_size=batch,
        class_mode='binary',
        seed=42,
        subset='validation')  # Set as validation data

    # Create a separate ImageDataGenerator for test data
    test_datagen = ImageDataGenerator(rescale=1./255)

    # Test data generator
    test_generator = test_datagen.flow_from_directory(
        dataset_path,
        target_size=tar,
        batch_size=batch,
        seed=42,
        class_mode='binary')  # No subset for testing
    
   

    # Paths to save splits
    base_dir = f'/app/data/Datasets/augmented_{classez[0]}_and_{classez[1]}-split'  # "G:\My Drive\Datasets\Trashnet-resized-split"
    train_dir = os.path.join(base_dir, 'train')
    val_dir = os.path.join(base_dir, 'val')
    test_dir = os.path.join(base_dir, 'test')

    create_dir(base_dir)
    create_dir(train_dir)
    create_dir(val_dir)
    create_dir(test_dir)

    for class_name in classez: #, 'cardboard', 'metal', 'paper', 'plastic', 'trash']:
        # Create class directories
        create_dir(os.path.join(train_dir, class_name))
        create_dir(os.path.join(val_dir, class_name))
        create_dir(os.path.join(test_dir, class_name))

        # List all files in the class directory
        class_files = os.listdir(os.path.join(dataset_path, class_name))
        train_files, test_files = train_test_split(class_files, test_size=0.2, random_state=42)  # 20% for testing
        train_files, val_files = train_test_split(train_files, test_size=0.25, random_state=42)  # 20% of remaining for validation (0.25 * 0.8 = 0.2)

        # Copy files to respective directories
        for file in train_files:
            shutil.copy(os.path.join(dataset_path, class_name, file), os.path.join(train_dir, class_name, file))

        for file in val_files:
            shutil.copy(os.path.join(dataset_path, class_name, file), os.path.join(val_dir, class_name, file))

        for file in test_files:
            shutil.copy(os.path.join(dataset_path, class_name, file), os.path.join(test_dir, class_name, file))
        
    train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=tar,
    batch_size=batch,
    seed=42,
    class_mode='binary'
    # ,color_mode='grayscale'
    )

    validation_generator = datagen.flow_from_directory(
        val_dir,
        target_size=tar,
        batch_size=batch,
        seed=42,
        class_mode='binary'
        # ,color_mode='grayscale'
        )

    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=tar,
        batch_size=batch,
        seed=42,
        class_mode='binary'
        # ,color_mode='grayscale'
        )

    img_shape = (224,224,3)
    img_size = tar
    batch_size = batch #16 #32
    num_classes = 2
    log_file = '/app/data/code/log_final.csv'

    # DenseNet201 # MobileNetV3 # EfficientNetB0
    base_model = tf.keras.applications.DenseNet201(input_shape=img_shape,
                                                include_top=False,
                                                weights='imagenet')

    model_name = base_model.name #'EfficientNetB0' # 'DenseNet201' # 'MobileNetV2'
    # print(model_name)

    base_model.trainable = False
    for layer in base_model.layers[-6:]:
        layer.trainable = True
        
    class AccuracyLogger(Callback):
        def __init__(self,   log_file, model_name, img_shape, batch_size, num_classes):
            super().__init__()
            self.log_file = log_file
            self.model_name = model_name
            self.img_shape = img_shape
            self.batch_size = batch_size
            self.num_classes = num_classes
            
            # self.summary = summary

            # Write model and dataset information at the beginning of the log file
            with open(self.log_file, 'a') as f:
                f.write(f"\nModel: {self.model_name}\n")
                # f.write(f"Model summary:{self.summary}\n")
                f.write(f"Image shape: {self.img_shape}\n")
                f.write(f"Batch size: {self.batch_size}\n")
                f.write(f"Dataset: {dataset_path}\n")
                f.write(f"Number of classes: {self.num_classes}\n\n")
                # f.write('Epoch,Accuracy,Validation Accuracy, Loss, Validation Loss\n')
                f.flush()

        # def on_epoch_end(self, epoch, logs=None):
        #     logs = logs or {}
        #     accuracy = logs.get('accuracy')
        #     val_accuracy = logs.get('val_accuracy')
        #     loss = logs.get('loss')
        #     val_loss = logs.get('val_loss')

        #     # Write the epoch, accuracy, and validation accuracy to the log file
        #     with open(self.log_file, 'a') as f:
        #         f.write(f"{epoch + 1},{accuracy},{val_accuracy},{loss},{val_loss}\n")
        #         f.flush()

    model = tf.keras.Sequential([base_model,
                                #  tf.keras.layers.Flatten(),
                                tf.keras.layers.GlobalAveragePooling2D(),
                                #  tf.keras.layers.Dense(batch, activation="relu"),
                                tf.keras.layers.Dropout(0.2),
                                tf.keras.layers.Dense(1, activation="sigmoid")
                                #  tf.keras.layers.Dense(2, activation="softmax")
                                ])

    lr_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                    patience=2, 
                                    factor=0.3, #0.5,
                                    verbose=1, 
                                    min_lr=0.01) #0.01)
    model.compile(optimizer = 'sgd',
                loss = 'binary_crossentropy',
                metrics = ['accuracy'])

    #Train for 10 epochs
    steps_per_epoch=int(len(train_generator)/batch_size)
    validation_steps = validation_generator.samples // validation_generator.batch_size

    # Train the model
    history = model.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        validation_data=validation_generator,
        callbacks=[AccuracyLogger(  log_file, model_name, img_shape, batch_size, num_classes),
                    EarlyStopping(patience=5)],
        epochs=100,
        verbose=1
    )

    def plot_graphs(history, batch_size):
        fig, axs = plt.subplots(1, 2, figsize=(12, 5))  # 1 row, 2 columns

        # Plot accuracy
        axs[0].plot(history.history['accuracy'])
        axs[0].plot(history.history['val_accuracy'])
        axs[0].set_title("Model Accuracy")
        axs[0].set_xlabel("Epochs")
        axs[0].set_ylabel("Accuracy")
        axs[0].legend(['Train', 'Validation'])

        # Plot loss
        axs[1].plot(history.history['loss'])
        axs[1].plot(history.history['val_loss'])
        axs[1].set_title("Model Loss")
        axs[1].set_xlabel("Epochs")
        axs[1].set_ylabel("Loss")
        axs[1].legend(['Train', 'Validation'])

        plt.tight_layout()  # Adjust layout to make sure plots don't overlap

        # Ensure the save directory exists
        save_dir = '/app/data/graphs/'
        os.makedirs(save_dir, exist_ok=True)

        # Create the filename using the dataset path and batch size
        dataset_name = os.path.basename(dataset_path[29:])  # filter out '/app/data/Datasets/' 
        filename = f"{dataset_name}_100epoch_batch{batch_size}_{model_name}.png"

        # Save the figure
        save_path = os.path.join(save_dir, filename)
        plt.savefig(save_path)
        print(f"Graph saved at: {save_path}")

        plt.show()

    # plot
    plot_graphs(history, batch)

    loss, accuracy = model.evaluate(test_generator, verbose=False)
    # Predict on test data
    y_pred = model.predict(test_generator)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true = test_generator.classes

    # Calculate precision, recall, and F1 score
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred_classes, average='weighted')

    # Calculate mAP
    # Note: This assumes binary classification. For multi-class, you'll need to modify this.
    mAP = average_precision_score(y_true, y_pred)

    # Create confusion matrix
    cm = confusion_matrix(y_true, y_pred_classes)

    # Log the metrics
    with open(log_file, 'a') as f:
        f.write(f"Testing Accuracy: {accuracy:.4f}\n")
        f.write(f"Precision: {precision:.4f}\n")
        f.write(f"Recall: {recall:.4f}\n")
        f.write(f"F1 Score: {f1:.4f}\n")
        f.write(f"mAP: {mAP:.4f}\n")
        f.flush()

    # Plot and save confusion matrix
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(f'{setClass[0]}confusion_matrix.png')
    plt.close()

    # Print metrics
    # print(f"Testing Accuracy: {accuracy:.4f}")
    # print(f"Precision: {precision:.4f}")
    # print(f"Recall: {recall:.4f}")
    # print(f"F1 Score: {f1:.4f}")
    # print(f"mAP: {mAP:.4f}")
    print(f"Confusion Matrix saved as '{setClass[0]}confusion_matrix.png'")

    # Save model
    model.save(f"/app/data/{model_name}_{setClass[0]}.keras")
    print(f"{setClass[0]} trained and tested successfully!")
print("Training complete!")


2024-10-07 12:24:43.120455: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-07 12:24:43.130293: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-07 12:24:43.141360: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-07 12:24:43.144686: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-07 12:24:43.153700: I tensorflow/core/platform/cpu_feature_guar

Found 3447 images belonging to 2 classes.
Found 861 images belonging to 2 classes.
Found 4308 images belonging to 2 classes.
Found 3032 images belonging to 2 classes.
Found 1020 images belonging to 2 classes.
Found 1019 images belonging to 2 classes.


I0000 00:00:1728303885.268942    2169 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728303885.273662    2169 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728303885.273835    2169 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728303885.274513    2169 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Epoch 1/100


I0000 00:00:1728303896.919368    2322 service.cc:146] XLA service 0x70eb740030f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1728303896.919392    2322 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce GTX 1050 Ti, Compute Capability 6.1
2024-10-07 12:24:57.361663: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-07 12:24:59.416567: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8906
2024-10-07 12:25:11.361789: W external/local_tsl/tsl/framework/bfc_allocator.cc:291] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.53GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


...

11/11 ━━━━━━━━━━━━━━━━━━━━ 70s 4s/step - accuracy: 0.5355 - loss: 0.7558 - val_accuracy: 0.5990 - val_loss: 0.6545
Epoch 2/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 30s 3s/step - accuracy: 0.6160 - loss: 0.6882 - val_accuracy: 0.7284 - val_loss: 0.5645
Epoch 3/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 8s 839ms/step - accuracy: 0.6768 - loss: 0.5899 - val_accuracy: 0.7833 - val_loss: 0.4999
Epoch 4/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 9s 839ms/step - accuracy: 0.7208 - loss: 0.5263 - val_accuracy: 0.7304 - val_loss: 0.5055
Epoch 5/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 9s 840ms/step - accuracy: 0.7441 - loss: 0.5216 - val_accuracy: 0.8147 - val_loss: 0.4538
Epoch 6/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 9s 841ms/step - accuracy: 0.8089 - loss: 0.4738 - val_accuracy: 0.8235 - val_loss: 0.4209
Epoch 7/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 9s 841ms/step - accuracy: 0.7661 - loss: 0.4861 - val_accuracy: 0.8363 - val_loss: 0.4186
Epoch 8/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 9s 841ms/step - accuracy: 0.7378 - loss: 0.5110 - val_accuracy: 0.8461 - v

2024-10-07 12:28:35.782202: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-10-07 12:28:35.782423: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]


11/11 ━━━━━━━━━━━━━━━━━━━━ 8s 750ms/step - accuracy: 0.8409 - loss: 0.4030 - val_accuracy: 0.8804 - val_loss: 0.3140
Epoch 19/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 9s 842ms/step - accuracy: 0.8143 - loss: 0.3962 - val_accuracy: 0.8735 - val_loss: 0.3403
Epoch 20/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 9s 843ms/step - accuracy: 0.9441 - loss: 0.2706 - val_accuracy: 0.8814 - val_loss: 0.3156
Epoch 21/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 8s 836ms/step - accuracy: 0.8882 - loss: 0.3386 - val_accuracy: 0.8853 - val_loss: 0.3024
Epoch 22/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 9s 842ms/step - accuracy: 0.8937 - loss: 0.3180 - val_accuracy: 0.8775 - val_loss: 0.3046
Epoch 23/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 9s 843ms/step - accuracy: 0.8797 - loss: 0.3248 - val_accuracy: 0.8922 - val_loss: 0.2927
Epoch 24/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 9s 842ms/step - accuracy: 0.8813 - loss: 0.3163 - val_accuracy: 0.8539 - val_loss: 0.3665
Epoch 25/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 9s 842ms/step - accuracy: 0.8464 - loss: 0.3462 - val_accuracy:

2024-10-07 12:31:08.729957: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]


11/11 ━━━━━━━━━━━━━━━━━━━━ 8s 752ms/step - accuracy: 0.8438 - loss: 0.3988 - val_accuracy: 0.9127 - val_loss: 0.2495
Epoch 37/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 9s 845ms/step - accuracy: 0.8872 - loss: 0.2788 - val_accuracy: 0.9167 - val_loss: 0.2468
Epoch 38/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 9s 845ms/step - accuracy: 0.9455 - loss: 0.2211 - val_accuracy: 0.9157 - val_loss: 0.2463
Epoch 39/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 9s 846ms/step - accuracy: 0.8588 - loss: 0.2995 - val_accuracy: 0.9147 - val_loss: 0.2451
Epoch 40/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 9s 845ms/step - accuracy: 0.9218 - loss: 0.2543 - val_accuracy: 0.9157 - val_loss: 0.2405
Epoch 41/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 9s 846ms/step - accuracy: 0.8650 - loss: 0.3163 - val_accuracy: 0.9225 - val_loss: 0.2404
Epoch 42/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 9s 845ms/step - accuracy: 0.8950 - loss: 0.2803 - val_accuracy: 0.9108 - val_loss: 0.2533
Epoch 43/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 9s 845ms/step - accuracy: 0.8251 - loss: 0.3837 - val_accuracy: